In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS


In [3]:
llm = AzureChatOpenAI(
    api_key=azure_openai_api_key,  # Azure OpenAI API Key
    api_version="2023-12-01-preview",  # API version (you can adjust it as needed)
    azure_endpoint=azure_openai_endpoint,  # Azure OpenAI endpoint
    deployment_name=deployment_name,  # Ensure deployment_name is the correct one you configured in Azure
    temperature=0  # Optional: Adjust the temperature for randomness in responses
)

C:\Users\antoa\AppData\Local\Temp\ipykernel_4884\1637258887.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [ ]:




# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chargement des documents
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    documents = []

# Création et sauvegarde du vecteur
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1, "filter": {"saveur": ["fraise", "framboise", "fruit_rouge"]}}
)

# Définition du prompt
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Pipeline de question-réponse
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne
try:
    question ='Propose moi un produit avec lime'
    response = qa_chain.invoke(question)
    
    print(response)
    
except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")

C:\Users\antoa\AppData\Local\Temp\ipykernel_29052\1118339573.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Nombre de documents chargés : 100
Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une contenance de 100ml. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni. Ce produit est surboosté en arômes et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).


benchmark temps

In [6]:
import time
# Fonction pour mesurer le temps de recommandation
def benchmark_recommendation(question):
    start_time = time.time()  # Temps de départ
    
    try:
        # Invocation de la chaîne de question-réponse
        response = qa_chain.invoke(question)
        print(response)  # Affichage de la réponse
    except Exception as e:
        print(f"Erreur lors de l'invocation : {e}")
    
    end_time = time.time()  # Temps de fin
    print(f"Temps de traitement : {end_time - start_time:.4f} secondes")


In [9]:
# Test de la chaîne de question-réponse avec la requête
question = 'Propose moi un produit avec lime'
benchmark_recommendation(question)

Je te propose le produit "Frozen Lemon And Lime 100ml Ice Chuffed". C'est un e-liquide aux saveurs fraîches de citrons jaunes et verts, avec une touche de lime. Il est fabriqué par la marque Chuffed, originaire du Royaume-Uni, et est disponible en format de 100ml. Ce produit est surboosté en arômes et peut être ajusté avec des boosters de nicotine selon tes besoins. Voici le lien pour plus de détails : [Frozen Lemon And Lime 100ml Ice Chuffed](https://www.aromes-et-liquides.fr/e-liquide-chuffed/14228-frozen-lemon-and-lime-100ml-ice.html).
Temps de traitement : 2.3278 secondes


2. Mesurer le Temps d'Exécution - FAISS + Embeddings

In [10]:
# Chargement des documents
loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
documents = loader.load()

# Création du vecteur FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

# Mesure du temps d'exécution
start_time = time.time()
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1})
retrieved_docs = retriever.invoke("Saveur cassis lime")
end_time = time.time()

# Affichage des résultats et du temps d'exécution
print(f"Temps d'exécution FAISS : {end_time - start_time:.4f} secondes")
print(retrieved_docs[:5])

Temps d'exécution FAISS : 0.0158 secondes
[Document(metadata={'source': 'rag_csv.csv', 'row': 69}, page_content="index: 69\nurl: https://www.aromes-et-liquides.fr/e-liquide-cirkus-vdlv/4806-e-liquide-cirkus-cassis-frais-par-vdlv.html\nnom_produit: Cassis Frais Cirkus VDLV\nimg_produit: https://assets.aromes-et-liquides.fr/55001-thickbox_default/e-liquide-cirkus-cassis-frais-par-vdlv.jpg\nprix_produit: 5.9\ncontenance: 10ml\npg_vg: 50/50\norigine: France\nfrais: Oui\nsurbooste: Non\nsaveur: cassis, lime, vin, menthol\ndescription: Retrouvez dessaveurs de cassis et de mentholdans un e-liquideprêt à l'emploifrais et fruité signé Vincent Dans Les Vapes. Le Cassis Frais de la gamme Cirkus par VDLV est un e-liquide français composé d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG) pour taux de 50PG/50VG. Il est proposé en fioles de 10ml souples à embouts fins pour simplifier le remplissage de toutes les cigarettes électroniques et il est disponible en plusieurs ta

3. Mesurer la pertinence des réponses 

In [12]:
from sentence_transformers import SentenceTransformer, util

# Charger le modèle d'évaluation
evaluation_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def evaluate_relevance(question, response):
    # Encoder la question et la réponse
    question_embedding = evaluation_model.encode(question, convert_to_tensor=True)
    response_embedding = evaluation_model.encode(response, convert_to_tensor=True)

    # Calculer la similarité cosinus
    similarity_score = util.pytorch_cos_sim(question_embedding, response_embedding).item()

    return similarity_score

# Test
question = "Propose-moi un produit avec lime"
response = qa_chain.invoke(question)

score = evaluate_relevance(question, response)
print(f"🔹 Similarité entre la question et la réponse : {score:.4f}")


🔹 Similarité entre la question et la réponse : 0.5932


In [18]:

docs = retriever.invoke('Je veux un produit avec lime')
print(type(docs))

<class 'list'>


In [7]:
llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_endpoint,
                        model=deployment_name,
                        temperature=0.9
                        )

In [8]:
result = await vectorstore.asimilarity_search(
    'eliquide lime',
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1},
    k=2)

result

[Document(metadata={'source': 'rag_csv.csv', 'row': 23}, page_content="index: 23\nurl: https://www.aromes-et-liquides.fr/e-liquide-eliquid-france/2525-exotic-eliquid-france.html\nnom_produit: Exotic Eliquid France\nimg_produit: https://assets.aromes-et-liquides.fr/54548-thickbox_default/exotic-eliquid-france.jpg\nprix_produit: 4.9\ncontenance: 10ml\npg_vg: 50/50\norigine: France\nfrais: Non\nsurbooste: Non\nsaveur: mangue, fruit de la passion, banane, lime, ananas, parfait\ndescription: L'Exotic par Eliquid France est fabriqué en France, et reproduit les saveursfruitéesd'un ananas, de mangue, de banane ou encore de fruit de la passion. Sa recetteprête à vaperest élaborée à partir d'arômes alimentaires, de propylène glycol (PG), et de glycérine végétale (VG), dans unratio 50PG/50VG. Il vous est proposé en 4 taux de nicotine (0, 3, 6, et 12mg/ml) afin de satisfaire un grand nombre de vapoteurs. Vous pouvez ainsi profiter d'un équilibre parfait entre restitution de saveurs et diffusion de

In [ ]:




# Configuration des embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chargement des documents
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    documents = loader.load()
    print(f"Nombre de documents chargés : {len(documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    documents = []

# Création et sauvegarde du vecteur
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

# Définition du prompt
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# Pipeline de question-réponse
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test de la chaîne
try:
    question ='Propose moi un produit avec lime'
    response = qa_chain.invoke(question)
    
    print(response)
    
except Exception as e:
    print(f"Erreur lors de l'invocation : {e}")

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.schema import Document

# Charger le modèle d'embeddings Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Charger les documents à partir du CSV avec gestion des métadonnées
try:
    loader = CSVLoader(file_path='rag_csv.csv', encoding='utf-8')
    raw_documents = loader.load()
    print(f"Nombre de documents chargés : {len(raw_documents)}")
except Exception as e:
    print(f"Erreur lors du chargement des documents : {e}")
    raw_documents = []

# Transformer les documents pour inclure la colonne "saveur" dans les métadonnées
documents = []
for doc in raw_documents:
    metadata = doc.metadata  # Récupérer les métadonnées existantes
    if "saveur" in metadata:  # Vérifier si la colonne existe
        metadata["saveur"] = metadata["saveur"]  # Ajouter "saveur" aux métadonnées
    documents.append(Document(page_content=doc.page_content, metadata=metadata))

# Création et sauvegarde du vecteur FAISS avec métadonnées
try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store enregistré avec succès !")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur avec prise en charge des métadonnées
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

# Définition du retriever avec seuil de similarité
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)

print("FAISS vector store chargé et prêt à l'emploi !")



C:\Users\antoa\AppData\Local\Temp\ipykernel_4884\2342884906.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



Nombre de documents chargés : 100
Vector store enregistré avec succès !
FAISS vector store chargé et prêt à l'emploi !


In [5]:
retriever.invoke("Saveur cassis lime")

[Document(metadata={'source': 'rag_csv.csv', 'row': 69}, page_content="index: 69\nurl: https://www.aromes-et-liquides.fr/e-liquide-cirkus-vdlv/4806-e-liquide-cirkus-cassis-frais-par-vdlv.html\nnom_produit: Cassis Frais Cirkus VDLV\nimg_produit: https://assets.aromes-et-liquides.fr/55001-thickbox_default/e-liquide-cirkus-cassis-frais-par-vdlv.jpg\nprix_produit: 5.9\ncontenance: 10ml\npg_vg: 50/50\norigine: France\nfrais: Oui\nsurbooste: Non\nsaveur: cassis, lime, vin, menthol\ndescription: Retrouvez dessaveurs de cassis et de mentholdans un e-liquideprêt à l'emploifrais et fruité signé Vincent Dans Les Vapes. Le Cassis Frais de la gamme Cirkus par VDLV est un e-liquide français composé d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG) pour taux de 50PG/50VG. Il est proposé en fioles de 10ml souples à embouts fins pour simplifier le remplissage de toutes les cigarettes électroniques et il est disponible en plusieurs taux de nicotine (3, 6 et 12mg/ml). Pour la 

In [10]:
import pandas as pd

# Charger le CSV
csv_path = "../../RGBD/table_produits/produits.csv"   # Remplace avec le chemin réel si besoin
df = pd.read_csv(csv_path, encoding='utf-8')

# Vérifier que la colonne "description" existe bien
if "description" not in df.columns:
    raise ValueError("⚠️ La colonne 'description' n'existe pas dans le CSV ! Vérifie le nom des colonnes.")

# Transformer chaque ligne en objet Document avec métadonnées
documents = []
for index, row in df.iterrows():
    metadata = {"source": csv_path, "row": index}

    if "saveur" in row:
        metadata["saveur"] = str(row["saveur"])  # Ajouter la saveur dans les métadonnées

    documents.append(Document(page_content=str(row["description"]), metadata=metadata))

print(f"Nombre de documents préparés : {len(documents)}")

# Création et sauvegarde du FAISS vector store avec métadonnées
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

try:
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local('faiss_vector_store')
    print("Vector store FAISS sauvegardé avec succès.")
except Exception as e:
    print(f"Erreur lors de la création du vectorstore : {e}")

# Chargement du vecteur
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1}
)
print("🚀 FAISS vector store chargé et prêt à l'emploi !")

# Tester une recherche
query = "Quel e-liquide a un goût fruité ?"
docs = retriever.get_relevant_documents(query)

for doc in docs:
    print(f"Texte : {doc.page_content}")
    print(f"Saveur : {doc.metadata.get('saveur', 'Non spécifiée')}")
    print("-" * 50)


Nombre de documents préparés : 1531
Vector store FAISS sauvegardé avec succès.
🚀 FAISS vector store chargé et prêt à l'emploi !
Texte : Le e-liquide Cola Cherry Candy est fabriqué en France par Prestige Fruits (Vape Connection) pour sa gamme Sweety Fruits. Il reproduit toutes les saveurs d'un bonbon au cola et à la cerise. Composé d'arômes, de propylène glycol et de glycérine végétale, il est présenté en 50PG/50VG pour un bon compromis hit en gorge, saveurs et vapeur et une excellente compatibilité avec vos matériels e-cig. Le e-liquide Cola Cherry Candy est conditionné en fiole de type PET 60ml, qui possède bien sûr un bouchon sécurisé et un embout fin très pratique. Remplie à hauteur de 50ml de e liquide non nicotiné, cette fiole pourra donc accueillir un booster de nicotine au besoin. Qui est Prestige Fruits ? Comme son nom l'indique, le fabriquant française s'est spécialisé dans la reproduction des saveurs de fruits à travers sa large gamme d'e-liquides. La collection Prestige Frui

In [17]:


# 🔹 Charger FAISS avec les métadonnées
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local('faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)



# 🔹 Fonction pour extraire la saveur demandée par l'utilisateur
def extract_flavor_from_query(query):
    prompt = f"Quelle est la saveur mentionnée dans cette phrase : '{query}' ? Donne seulement la saveur exacte, sans phrase supplémentaire."
    return llm.predict(prompt).strip().lower()  # Convertir en minuscule pour éviter les erreurs

# 🔹 Fonction pour récupérer les documents contenant cette saveur
def search_by_flavor(query):
    # 🎯 Extraire la saveur demandée avec le LLM
    flavor = extract_flavor_from_query(query)
    print(f"🔍 Saveur détectée : {flavor}")  # Debugging

    # 🛑 Récupérer tous les documents indexés dans FAISS
    all_docs = vectorstore.docstore._dict.values()

    # 🎯 Filtrer uniquement ceux qui contiennent cette saveur
    filtered_docs = [doc for doc in all_docs if flavor in doc.metadata.get("saveur", "").lower()]

    if not filtered_docs:
        return f"Aucun e-liquide trouvé avec la saveur '{flavor}'."

    # 🏆 Afficher les résultats trouvés
    results = "\n\n".join([f"📌 **Produit** : {doc.page_content}\n🔹 **Saveur** : {doc.metadata['saveur']}" for doc in filtered_docs])
    
    return f"✅ Voici les e-liquides correspondant à la saveur '{flavor}':\n\n{results}"



In [18]:
# 🎯 Tester la fonction
query = "Je veux un e-liquide à la fraise"
result = search_by_flavor(query)
print(result)

C:\Users\antoa\AppData\Local\Temp\ipykernel_4884\3194913576.py:10: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm.predict(prompt).strip().lower()  # Convertir en minuscule pour éviter les erreurs


🔍 Saveur détectée : fraise
✅ Voici les e-liquides correspondant à la saveur 'fraise':

📌 **Produit** : Découvrez une mono-saveur dédiée à la fraise des bois, dans un e-liquide fabriqué en France par PULP. Le produit est idéal pour débuter, déjàprêt à vaperet disponible en plusieurs taux de nicotine : 0, 3, 6 ou 12mg/ml. Le Fraise Sauvage est élaboré à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale à hauteur de 70PG et 30VG. Vous retrouverez ce e-liquide dans des flacons de 10ml en plastique PET avec embout fin. Ces fioles sont également équipées d'une sécurité enfant. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours avec le sens du détail. Ainsi, il vous présente aussi bien des e-liquides prêts à vaper, à booster, ou encore des concentrés DIY (Do It Yourself) aux saveurs finement reproduites et de diverses variétés. Nos conseils p